In [1]:
#!pip install textblob
#!pip install langdetect
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tianj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
import plotly.graph_objects as go
# from better_profanity import profanity
import string

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [3]:
songdata = pd.read_csv('./songdata.csv')

In [4]:
songdata.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df = pd.read_pickle("./genres.pkl")
df = pd.merge(songdata, df, on=['artist','artist'])
df = df[df.iloc[:,5:].any(axis = 1)]

In [6]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","[europop, swedish pop]",0,0,1,0,0,0,0
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","[europop, swedish pop]",0,0,1,0,0,0,0
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,"[europop, swedish pop]",0,0,1,0,0,0,0
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51489 entries, 0 to 57595
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             51489 non-null  object
 1   song               51489 non-null  object
 2   link               51489 non-null  object
 3   text               51489 non-null  object
 4   genres             51489 non-null  object
 5   rock               51489 non-null  int32 
 6   singer-songwriter  51489 non-null  int32 
 7   pop                51489 non-null  int32 
 8   metal              51489 non-null  int32 
 9   folk               51489 non-null  int32 
 10  country            51489 non-null  int32 
 11  hip hop / rap      51489 non-null  int32 
dtypes: int32(7), object(5)
memory usage: 3.7+ MB


We have a multilabel classification problem according to https://scikit-learn.org/stable/modules/multiclass.html

Multilabel- 

"A multiclass multioutput target where each output is binary. This may be represented as a 2d (dense) array or sparse matrix of integers, such that each column is a separate binary target, where positive labels are indicated with 1 and negative labels are usually -1 or 0. Sparse multilabel targets are not supported everywhere that dense multilabel targets are supported."

"Valid representation of multilabel y is either dense (or sparse) binary matrix of shape (n_samples, n_classes). Each column represents a class. The 1’s in each row denote the positive classes a sample has been labelled with."

Some text is in round brackets while some is in square brackets. I wanted to examine what that text looked like.

## Removing Non-English Words

### Removing Non-English Songs

Note: This takes a while to run. 

In [8]:
#This find the probability that the word is english
def get_eng_prob(text):
    detections = detect_langs(text)
    for detection in detections:
        if detection.lang == 'en':
            return detection.prob
    return 0

#finding the probability that the text is english
df['en_prob'] = df['text'].map(get_eng_prob)

print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51122
Number of songs that are not english: 367


In [9]:
#only selecting songs that have a probability of 0.5 or higher of being in english
df= df.loc[df['en_prob'] >= 0.5]

In [10]:
df.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,en_prob
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999998
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","[europop, swedish pop]",0,0,1,0,0,0,0,0.999996
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999997
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999997
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[europop, swedish pop]",0,0,1,0,0,0,0,0.999998


In [11]:
#Repeating above code to make sure that the non-english songs were dropped
print('Number of songs in english: {}'.format(sum(df['en_prob'] >= 0.5)))
print('Number of songs that are not english: {}'.format(sum(df['en_prob'] < 0.5)))

Number of songs in english: 51122
Number of songs that are not english: 0


In [12]:
df = df.drop(['en_prob'], axis=1)

## Splitting df into Train/Test
We split the initial dataframe into train/test sets before applying any text preprocessing. This is to avoid any unintentional influence of training sets on testing sets, as preprocessing tools performing imputations on the entire data set.

In [13]:
df_train, df_test = train_test_split(df, 
                                     test_size = 0.2,
                                     random_state = 0)

In [14]:
df_train.shape

(40897, 12)

In [15]:
df_test.shape

(10225, 12)

## Text PreProcessing - df Train

In [16]:
#examining text in round brackets
round_brackets = sum(list(df_train['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
#Number of round brackets:
print((len(round_brackets)))

43170


In [17]:
#Viewing some text in round brackets
#These just look like normal lyrics
random.seed(0)
random.choices(round_brackets, k=15)

['can I get a witness',
 'Keep it on the up',
 'Verse Two',
 'Ooh',
 'tell me do you-oh-oh-oh',
 'Ms. Hangover',
 'So low, low cool',
 'Suspicions',
 'ooh',
 'Bulbasaur!',
 "Can't explain",
 'For sure',
 'It was best for you and me',
 'Oh!',
 '[Mark:] I believe in angels ']

In [18]:
#examining text in square brackets
square_brackets = sum(list(df_train['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
#how many instances of square brackets?
print((len(square_brackets)))

20745


In [19]:
#re.sub(pattern, repl, string, count=0, flags=0)
# remove round brackets but not text within
df_train['text'] = df_train['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
df_train['text'] = df_train['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# remove line breaks
df_train['text'] = df_train['text'].map(lambda s: re.sub(r' \n|\n', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
df_train['text'].map(lambda s: re.sub(r' \n|\n', '', s))

40376    Celia, you're breaking my heart  \rYou're shak...
41151    Shock the monkey to life  \rShock the monkey t...
26956    When I picked you up for our first date baby  ...
32590    Do you or don't you want this to be your song?...
28655    Her love is a gun, her love is a blade, she's ...
                               ...                        
24085    You may talk of gin and beer  \rWhen you're st...
51808    I can feel the world upon my back  \rI got six...
47912    In view of the fact that there's a rumour goin...
49072    There's been some knocking  \rThere's been som...
2962     Oh we was dancing in a really big city  \rGot ...
Name: text, Length: 40897, dtype: object

### Stemming and Preprocessing

In [22]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
 
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):

    result = []
    #stopwords = list(gensim.parsing.preprocessing.STOPWORDS)
    #stopwords.pop(stopwords.index('no'))
    for token in gensim.utils.simple_preprocess(text):
        #if token not in set(stopwords):
        result.append(lemmatize_stemming(token))
    return ' '.join(result)

In [23]:
df_train['text_preprocessed'] = df_train['text'].map(preprocess)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed
40376,Paul Simon,Cecilia,/p/paul+simon/cecilia_20105972.html,"Celia, you're breaking my heart \rYou're shak...","[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,celia you re break my heart you re shake my co...
41151,Peter Gabriel,Shock The Monkey,/p/peter+gabriel/shock+the+monkey_20107488.html,Shock the monkey to life \rShock the monkey t...,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,shock the monkey to life shock the monkey to l...
26956,Keith Urban,Sweet Thing,/k/keith+urban/sweet+thing_20769753.html,When I picked you up for our first date baby ...,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,when pick you up for our first date babi well ...
32590,Marilyn Manson,Born Again,/m/marilyn+manson/born+again_20088983.html,Do you or don't you want this to be your song?...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,do you or don you want thi to be your song it ...
28655,Kiss,Killer,/k/kiss/killer_20079761.html,"Her love is a gun, her love is a blade, she's ...","[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,her love be gun her love be blade she killer b...


In [25]:
#Checking what the tokens column look like before removing stopwords
df_train['text_preprocessed'].head()

40376    celia you re break my heart you re shake my co...
41151    shock the monkey to life shock the monkey to l...
26956    when pick you up for our first date babi well ...
32590    do you or don you want thi to be your song it ...
28655    her love be gun her love be blade she killer b...
Name: text_preprocessed, dtype: object

In [26]:
#safety
df_train2 = df_train

## Tokenization

In [27]:
#w+ for whitespace
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
df_train['tokens'] = df_train['text_preprocessed'].map(tokenizer.tokenize)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
40376,Paul Simon,Cecilia,/p/paul+simon/cecilia_20105972.html,"Celia, you're breaking my heart \rYou're shak...","[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,celia you re break my heart you re shake my co...,"[celia, you, re, break, my, heart, you, re, sh..."
41151,Peter Gabriel,Shock The Monkey,/p/peter+gabriel/shock+the+monkey_20107488.html,Shock the monkey to life \rShock the monkey t...,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,shock the monkey to life shock the monkey to l...,"[shock, the, monkey, to, life, shock, the, mon..."
26956,Keith Urban,Sweet Thing,/k/keith+urban/sweet+thing_20769753.html,When I picked you up for our first date baby ...,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,when pick you up for our first date babi well ...,"[when, pick, you, up, for, our, first, date, b..."
32590,Marilyn Manson,Born Again,/m/marilyn+manson/born+again_20088983.html,Do you or don't you want this to be your song?...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,do you or don you want thi to be your song it ...,"[do, you, or, don, you, want, thi, to, be, you..."
28655,Kiss,Killer,/k/kiss/killer_20079761.html,"Her love is a gun, her love is a blade, she's ...","[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,her love be gun her love be blade she killer b...,"[her, love, be, gun, her, love, be, blade, she..."


### Removing stop words
Reference for more info: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [29]:
#setting stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tianj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#removing stop words
df_train['tokens'] = df_train['tokens'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
40376,Paul Simon,Cecilia,/p/paul+simon/cecilia_20105972.html,"Celia, you're breaking my heart \rYou're shak...","[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,celia you re break my heart you re shake my co...,"[celia, break, heart, shake, confid, daili, oh..."
41151,Peter Gabriel,Shock The Monkey,/p/peter+gabriel/shock+the+monkey_20107488.html,Shock the monkey to life \rShock the monkey t...,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,shock the monkey to life shock the monkey to l...,"[shock, monkey, life, shock, monkey, life, cov..."
26956,Keith Urban,Sweet Thing,/k/keith+urban/sweet+thing_20769753.html,When I picked you up for our first date baby ...,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,when pick you up for our first date babi well ...,"[pick, first, date, babi, well, pretti, blue, ..."
32590,Marilyn Manson,Born Again,/m/marilyn+manson/born+again_20088983.html,Do you or don't you want this to be your song?...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,do you or don you want thi to be your song it ...,"[want, thi, song, take, rebel, sing, along, th..."
28655,Kiss,Killer,/k/kiss/killer_20079761.html,"Her love is a gun, her love is a blade, she's ...","[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,her love be gun her love be blade she killer b...,"[love, gun, love, blade, killer, bitch, insan,..."


In [32]:
#checking out the data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40897 entries, 40376 to 2962
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             40897 non-null  object
 1   song               40897 non-null  object
 2   link               40897 non-null  object
 3   text               40897 non-null  object
 4   genres             40897 non-null  object
 5   rock               40897 non-null  int32 
 6   singer-songwriter  40897 non-null  int32 
 7   pop                40897 non-null  int32 
 8   metal              40897 non-null  int32 
 9   folk               40897 non-null  int32 
 10  country            40897 non-null  int32 
 11  hip hop / rap      40897 non-null  int32 
 12  text_preprocessed  40897 non-null  object
 13  tokens             40897 non-null  object
dtypes: int32(7), object(7)
memory usage: 3.6+ MB


In [33]:
#turning the stems back into untokenized structrure so that we can run count vector on it
df_train['stem_str'] = df_train['tokens'].map(lambda lst: ' '.join(lst))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
df_train.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str
40376,Paul Simon,Cecilia,/p/paul+simon/cecilia_20105972.html,"Celia, you're breaking my heart \rYou're shak...","[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,celia you re break my heart you re shake my co...,"[celia, break, heart, shake, confid, daili, oh...",celia break heart shake confid daili oh cecili...
41151,Peter Gabriel,Shock The Monkey,/p/peter+gabriel/shock+the+monkey_20107488.html,Shock the monkey to life \rShock the monkey t...,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,shock the monkey to life shock the monkey to l...,"[shock, monkey, life, shock, monkey, life, cov...",shock monkey life shock monkey life cover run ...
26956,Keith Urban,Sweet Thing,/k/keith+urban/sweet+thing_20769753.html,When I picked you up for our first date baby ...,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,when pick you up for our first date babi well ...,"[pick, first, date, babi, well, pretti, blue, ...",pick first date babi well pretti blue eye driv...
32590,Marilyn Manson,Born Again,/m/marilyn+manson/born+again_20088983.html,Do you or don't you want this to be your song?...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,do you or don you want thi to be your song it ...,"[want, thi, song, take, rebel, sing, along, th...",want thi song take rebel sing along thi art we...
28655,Kiss,Killer,/k/kiss/killer_20079761.html,"Her love is a gun, her love is a blade, she's ...","[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,her love be gun her love be blade she killer b...,"[love, gun, love, blade, killer, bitch, insan,...",love gun love blade killer bitch insan deal pa...


### Vectorization

In [35]:
df_train['tokens2']=df_train.tokens.apply(lambda x:str(x).strip('[]'))
df_train.head()                  

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str,tokens2
40376,Paul Simon,Cecilia,/p/paul+simon/cecilia_20105972.html,"Celia, you're breaking my heart \rYou're shak...","[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,celia you re break my heart you re shake my co...,"[celia, break, heart, shake, confid, daili, oh...",celia break heart shake confid daili oh cecili...,"'celia', 'break', 'heart', 'shake', 'confid', ..."
41151,Peter Gabriel,Shock The Monkey,/p/peter+gabriel/shock+the+monkey_20107488.html,Shock the monkey to life \rShock the monkey t...,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,shock the monkey to life shock the monkey to l...,"[shock, monkey, life, shock, monkey, life, cov...",shock monkey life shock monkey life cover run ...,"'shock', 'monkey', 'life', 'shock', 'monkey', ..."
26956,Keith Urban,Sweet Thing,/k/keith+urban/sweet+thing_20769753.html,When I picked you up for our first date baby ...,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,when pick you up for our first date babi well ...,"[pick, first, date, babi, well, pretti, blue, ...",pick first date babi well pretti blue eye driv...,"'pick', 'first', 'date', 'babi', 'well', 'pret..."
32590,Marilyn Manson,Born Again,/m/marilyn+manson/born+again_20088983.html,Do you or don't you want this to be your song?...,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,do you or don you want thi to be your song it ...,"[want, thi, song, take, rebel, sing, along, th...",want thi song take rebel sing along thi art we...,"'want', 'thi', 'song', 'take', 'rebel', 'sing'..."
28655,Kiss,Killer,/k/kiss/killer_20079761.html,"Her love is a gun, her love is a blade, she's ...","[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,her love be gun her love be blade she killer b...,"[love, gun, love, blade, killer, bitch, insan,...",love gun love blade killer bitch insan deal pa...,"'love', 'gun', 'love', 'blade', 'killer', 'bit..."


we want to keep: 

artists

song title

tokens2 (name it lyrics)

genre x7

## TF-IDF

In [36]:
tfidf = TfidfVectorizer()
veclyrics_tfidf_train = tfidf.fit_transform(df_train['tokens2'])

In [37]:
type(veclyrics_tfidf_train)

scipy.sparse.csr.csr_matrix

In [41]:
veclyrics_tfidf_train.shape

(40897, 42176)

## Text PreProcess - df Test

In [42]:
#examining text in round brackets
round_brackets = sum(list(df_test['text'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
#Number of round brackets:
print((len(round_brackets)))

11386


In [43]:
#Viewing some text in round brackets
#These just look like normal lyrics
random.seed(0)
random.choices(round_brackets, k=15)

['trying to bend the truth',
 'She needs it bad, real real bad',
 'they love ya boy',
 'You are',
 '2X',
 'Oh',
 'It could happen anytime',
 'Stand it',
 'uhh hun',
 "You got Shaolin, don't shoot the shit out",
 'from the relief you bring.',
 'Yeah, yeah, yeah',
 'watch them stare',
 'Does this make me look fat?',
 'Follow you']

In [44]:
#examining text in square brackets
square_brackets = sum(list(df_test['text'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
#how many instances of square brackets?
print((len(square_brackets)))

5026


In [45]:
#re.sub(pattern, repl, string, count=0, flags=0)
# remove round brackets but not text within
df_test['text'] = df_test['text'].map(lambda s: re.sub(r'\(|\)', '', s))

# remove square brackest and text within
df_test['text'] = df_test['text'].map(lambda s: re.sub(r'\[(.*?)\] ', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
# remove line breaks
df_test['text'] = df_test['text'].map(lambda s: re.sub(r' \n|\n', '', s))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Stemming and Preprocessing

In [47]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
 
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):

    result = []
    #stopwords = list(gensim.parsing.preprocessing.STOPWORDS)
    #stopwords.pop(stopwords.index('no'))
    for token in gensim.utils.simple_preprocess(text):
        #if token not in set(stopwords):
        result.append(lemmatize_stemming(token))
    return ' '.join(result)

In [48]:
df_test['text_preprocessed'] = df_test['text'].map(preprocess)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed
41486,Phish,Birds Of A Feather,/p/phish/birds+of+a+feather_20108434.html,It's easy sometimes when you just coast along ...,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,it easi sometim when you just coast along but ...
26066,Judds,Love Can Build A Bridge,/j/judds/love+can+build+a+bridge_20076356.html,I'd gladly walk across the desert \rWith no s...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,gladli walk across the desert with no shoe upo...
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...
18081,Glee,A Hard Day's Night,/g/glee/a+hard+days+night_21069138.html,It's been a hard day's night \rAnd I've been ...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,it be hard day night and ve be work like dog i...
19560,Hank Snow,"Keep Your Promise, Willie Thomas",/h/hank+snow/keep+your+promise+willie+thomas+f...,"Keep your promise, Willie Thomas \rWe pledged...","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,keep your promis willi thoma we pledg our love...


In [50]:
#Checking what the tokens column look like before removing stopwords
df_test['text_preprocessed'].head()

41486    it easi sometim when you just coast along but ...
26066    gladli walk across the desert with no shoe upo...
135      be as young as feel alway be alway est la vie ...
18081    it be hard day night and ve be work like dog i...
19560    keep your promis willi thoma we pledg our love...
Name: text_preprocessed, dtype: object

In [51]:
#safety
df_test2 = df_test

## Tokenization

In [52]:
#w+ for whitespace
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

#creating a new column called tokens
df_test['tokens'] = df_test['text_preprocessed'].map(tokenizer.tokenize)

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
41486,Phish,Birds Of A Feather,/p/phish/birds+of+a+feather_20108434.html,It's easy sometimes when you just coast along ...,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,it easi sometim when you just coast along but ...,"[it, easi, sometim, when, you, just, coast, al..."
26066,Judds,Love Can Build A Bridge,/j/judds/love+can+build+a+bridge_20076356.html,I'd gladly walk across the desert \rWith no s...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,gladli walk across the desert with no shoe upo...,"[gladli, walk, across, the, desert, with, no, ..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[be, as, young, as, feel, alway, be, alway, es..."
18081,Glee,A Hard Day's Night,/g/glee/a+hard+days+night_21069138.html,It's been a hard day's night \rAnd I've been ...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,it be hard day night and ve be work like dog i...,"[it, be, hard, day, night, and, ve, be, work, ..."
19560,Hank Snow,"Keep Your Promise, Willie Thomas",/h/hank+snow/keep+your+promise+willie+thomas+f...,"Keep your promise, Willie Thomas \rWe pledged...","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,keep your promis willi thoma we pledg our love...,"[keep, your, promis, willi, thoma, we, pledg, ..."


### Removing stop words
Reference for more info: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [54]:
#removing stop words
df_test['tokens'] = df_test['tokens'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens
41486,Phish,Birds Of A Feather,/p/phish/birds+of+a+feather_20108434.html,It's easy sometimes when you just coast along ...,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,it easi sometim when you just coast along but ...,"[easi, sometim, coast, along, like, someth, al..."
26066,Judds,Love Can Build A Bridge,/j/judds/love+can+build+a+bridge_20076356.html,I'd gladly walk across the desert \rWith no s...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,gladli walk across the desert with no shoe upo...,"[gladli, walk, across, desert, shoe, upon, fee..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,..."
18081,Glee,A Hard Day's Night,/g/glee/a+hard+days+night_21069138.html,It's been a hard day's night \rAnd I've been ...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,it be hard day night and ve be work like dog i...,"[hard, day, night, work, like, dog, hard, day,..."
19560,Hank Snow,"Keep Your Promise, Willie Thomas",/h/hank+snow/keep+your+promise+willie+thomas+f...,"Keep your promise, Willie Thomas \rWe pledged...","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,keep your promis willi thoma we pledg our love...,"[keep, promis, willi, thoma, pledg, love, one,..."


In [56]:
#checking out the data
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10225 entries, 41486 to 41304
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             10225 non-null  object
 1   song               10225 non-null  object
 2   link               10225 non-null  object
 3   text               10225 non-null  object
 4   genres             10225 non-null  object
 5   rock               10225 non-null  int32 
 6   singer-songwriter  10225 non-null  int32 
 7   pop                10225 non-null  int32 
 8   metal              10225 non-null  int32 
 9   folk               10225 non-null  int32 
 10  country            10225 non-null  int32 
 11  hip hop / rap      10225 non-null  int32 
 12  text_preprocessed  10225 non-null  object
 13  tokens             10225 non-null  object
dtypes: int32(7), object(7)
memory usage: 918.7+ KB


In [57]:
#turning the stems back into untokenized structrure so that we can run count vector on it
df_test['stem_str'] = df_test['tokens'].map(lambda lst: ' '.join(lst))

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
df_test.head()

,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str
41486,Phish,Birds Of A Feather,/p/phish/birds+of+a+feather_20108434.html,It's easy sometimes when you just coast along ...,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,it easi sometim when you just coast along but ...,"[easi, sometim, coast, along, like, someth, al...",easi sometim coast along like someth alway see...
26066,Judds,Love Can Build A Bridge,/j/judds/love+can+build+a+bridge_20076356.html,I'd gladly walk across the desert \rWith no s...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,gladli walk across the desert with no shoe upo...,"[gladli, walk, across, desert, shoe, upon, fee...",gladli walk across desert shoe upon feet share...
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,...",young feel alway alway est la vie est la vie e...
18081,Glee,A Hard Day's Night,/g/glee/a+hard+days+night_21069138.html,It's been a hard day's night \rAnd I've been ...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,it be hard day night and ve be work like dog i...,"[hard, day, night, work, like, dog, hard, day,...",hard day night work like dog hard day night sl...
19560,Hank Snow,"Keep Your Promise, Willie Thomas",/h/hank+snow/keep+your+promise+willie+thomas+f...,"Keep your promise, Willie Thomas \rWe pledged...","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,keep your promis willi thoma we pledg our love...,"[keep, promis, willi, thoma, pledg, love, one,...",keep promis willi thoma pledg love one day ben...


### Vectorization

In [59]:
df_test['tokens2']=df_test.tokens.apply(lambda x:str(x).strip('[]'))
df_test.head()   

C:\Users\tianj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,artist,song,link,text,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,text_preprocessed,tokens,stem_str,tokens2
41486,Phish,Birds Of A Feather,/p/phish/birds+of+a+feather_20108434.html,It's easy sometimes when you just coast along ...,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,it easi sometim when you just coast along but ...,"[easi, sometim, coast, along, like, someth, al...",easi sometim coast along like someth alway see...,"'easi', 'sometim', 'coast', 'along', 'like', '..."
26066,Judds,Love Can Build A Bridge,/j/judds/love+can+build+a+bridge_20076356.html,I'd gladly walk across the desert \rWith no s...,"[country, country dawn, country rock]",1,0,0,0,0,1,0,gladli walk across the desert with no shoe upo...,"[gladli, walk, across, desert, shoe, upon, fee...",gladli walk across desert shoe upon feet share...,"'gladli', 'walk', 'across', 'desert', 'shoe', ..."
135,Ace Of Base,C'est La Vie,/a/ace+of+base/cest+la+vie_10002096.html,"I am as young as I'm feeling \rI'm always 21,...","[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,be as young as feel alway be alway est la vie ...,"[young, feel, alway, alway, est, la, vie, est,...",young feel alway alway est la vie est la vie e...,"'young', 'feel', 'alway', 'alway', 'est', 'la'..."
18081,Glee,A Hard Day's Night,/g/glee/a+hard+days+night_21069138.html,It's been a hard day's night \rAnd I've been ...,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,it be hard day night and ve be work like dog i...,"[hard, day, night, work, like, dog, hard, day,...",hard day night work like dog hard day night sl...,"'hard', 'day', 'night', 'work', 'like', 'dog',..."
19560,Hank Snow,"Keep Your Promise, Willie Thomas",/h/hank+snow/keep+your+promise+willie+thomas+f...,"Keep your promise, Willie Thomas \rWe pledged...","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,keep your promis willi thoma we pledg our love...,"[keep, promis, willi, thoma, pledg, love, one,...",keep promis willi thoma pledg love one day ben...,"'keep', 'promis', 'willi', 'thoma', 'pledg', '..."


## Doc2Vec

In [60]:
tfidf = TfidfVectorizer()
veclyrics_tfidf_test = tfidf.fit_transform(df_test['tokens2'])

In [61]:
type(veclyrics_tfidf_test)

scipy.sparse.csr.csr_matrix

In [62]:
veclyrics_tfidf_test.shape

(10225, 22079)

## Save vectorized lyrics and responses for train and test sets

#### Pickle vectorized lyrics

In [63]:
pickle.dump(veclyrics_tfidf_train, open( "veclyrics_tfidf_train.pkl", "wb" ))

In [64]:
pickle.dump(veclyrics_tfidf_test, open( "veclyrics_tfidf_test.pkl", "wb" ))

#### Format and pickle genre response

In [65]:
df_train = df_train.drop(['link','text','text_preprocessed','stem_str','tokens'], axis = 1)
df_test = df_test.drop(['link','text','text_preprocessed','stem_str','tokens'], axis = 1)

In [66]:
df_train.rename(columns={'tokens2':'tokenstring'}, inplace=True)
df_test.rename(columns={'tokens2':'tokenstring'}, inplace=True)

In [67]:
df_train.head()

,artist,song,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,tokenstring
40376,Paul Simon,Cecilia,"[classic rock, folk, folk rock, mellow gold, p...",1,1,0,0,1,0,0,"'celia', 'break', 'heart', 'shake', 'confid', ..."
41151,Peter Gabriel,Shock The Monkey,"[album rock, art pop, art rock, classic rock, ...",1,0,1,0,0,0,0,"'shock', 'monkey', 'life', 'shock', 'monkey', ..."
26956,Keith Urban,Sweet Thing,"[australian country, australian pop, contempor...",0,0,1,0,0,1,0,"'pick', 'first', 'date', 'babi', 'well', 'pret..."
32590,Marilyn Manson,Born Again,"[alternative metal, industrial, industrial met...",1,0,0,1,0,0,0,"'want', 'thi', 'song', 'take', 'rebel', 'sing'..."
28655,Kiss,Killer,"[album rock, classic rock, glam rock, hard roc...",1,0,0,1,0,0,0,"'love', 'gun', 'love', 'blade', 'killer', 'bit..."


In [68]:
df_test.head()

,artist,song,genres,rock,singer-songwriter,pop,metal,folk,country,hip hop / rap,tokenstring
41486,Phish,Birds Of A Feather,"[blues rock, jam band, roots rock]",1,0,0,0,0,0,0,"'easi', 'sometim', 'coast', 'along', 'like', '..."
26066,Judds,Love Can Build A Bridge,"[country, country dawn, country rock]",1,0,0,0,0,1,0,"'gladli', 'walk', 'across', 'desert', 'shoe', ..."
135,Ace Of Base,C'est La Vie,"[bubblegum dance, dance pop, eurodance, europo...",1,0,1,0,0,0,0,"'young', 'feel', 'alway', 'alway', 'est', 'la'..."
18081,Glee,A Hard Day's Night,"[glee club, hollywood, post-teen pop, viral pop]",0,0,1,0,0,0,0,"'hard', 'day', 'night', 'work', 'like', 'dog',..."
19560,Hank Snow,"Keep Your Promise, Willie Thomas","[canadian country, canadian singer-songwriter,...",0,1,0,0,0,1,0,"'keep', 'promis', 'willi', 'thoma', 'pledg', '..."


In [69]:
df_train.to_pickle("response_tfidf_train.pkl")
df_test.to_pickle("response_tfidf_test.pkl")